### Importing the libraries:
We import the required libraries.

In [ ]:
!pip install pyspellchecker
!pip install langdetect
import nltk
nltk.download('wordnet')
nltk.download('words')

In [200]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import *
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from wordcloud import WordCloud
import re
import string
from collections import Counter
from spellchecker import SpellChecker
from langdetect import detect
from nltk.corpus import wordnet
from nltk.corpus import words

### Loading the dataset:
We use the pandas library to load the csv dataset.

In [201]:
data = pd.read_csv('sentiment_analysis.csv', encoding='latin-1', header=None)

### Initial Dataframe:

In [202]:
data.head()

,0,1,2,3,4,5
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [203]:
data.tail()

,0,1,2,3,4,5
1599995,4,2193601966,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,AmandaMarie1028,Just woke up. Having no school is the best fee...
1599996,4,2193601969,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,TheWDBoards,TheWDB.com - Very cool to hear old Walt interv...
1599997,4,2193601991,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,bpbabe,Are you ready for your MoJo Makeover? Ask me f...
1599998,4,2193602064,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,tinydiamondz,Happy 38th Birthday to my boo of alll time!!! ...
1599999,4,2193602129,Tue Jun 16 08:40:50 PDT 2009,NO_QUERY,RyanTrevMorris,happy #charitytuesday @theNSPCC @SparksCharity...


### Pre-processing steps and Transformations:

The columns' names for this dataset are not clear, so to fix it, we change the columns' names.

In [204]:
#Setting the column names
data.columns = ['sentiment', 'id', 'date', 'flag', 'user', 'text']

In [205]:
#Dropping columns we don't need
data = data.drop(columns= ['flag'])

In [206]:
#Checking for null values
data.isnull().sum()

sentiment    0
id           0
date         0
user         0
text         0
dtype: int64

This indicates that there are no null values in any of the columns of the dataframe.

In [207]:
#Changing contracted words into their root form.
def contractions(text):
    text = re.sub(r"won't", 'will not',text)
    text = re.sub(r"would't", 'would not',text)
    text = re.sub(r"could't", 'could not',text)
    text = re.sub(r'\'d', 'would',text)
    text = re.sub(r"can\'t", 'can not',text)
    text = re.sub(r"isn\'t", 'is not',text)
    text = re.sub(r"don\'t", 'do not',text)
    text = re.sub(r"n\'t", ' not', text)
    text = re.sub(r"\'re", ' are', text)
    text = re.sub(r"\'s", ' is', text)
    text = re.sub(r"\'ll", ' will', text)
    text = re.sub(r"\'t", ' not', text)
    text = re.sub(r"\'ve", ' have', text)
    text = re.sub(r"\'m", ' am', text)
    text = re.sub(r"shoulda", "should have", text)
    text = re.sub(r"gonna", "going to", text)
    text = re.sub(r"wanna", "want to", text)
    text = re.sub(r"wutcha", "what are you", text)
    text = re.sub(r"^im", "I am", text)
    return text

Defining the pre-processing steps and Transformations for the tweets in the text column.

In [208]:
def preprocess_text(text):
    # Lowercase the tweet text
    text = text.lower()
    
    # Perform contractions
    text = contractions(text)
    
    # Remove URLs from the tweet text
    text = re.sub(r'http\S+|www\.\S+|\w+.com', '', text)
    
    # Remove mentions from the tweet text
    text = re.sub(r'@[A-Za-z0-9]+', '', text)
    
    #Remove numbers from the tweet text
    text = re.sub(r'[0-9]+', '', text)
    
    #Remove one or two letter words from the tweet text
    text = re.sub(r"\b\w{1,2}\b", " ", text)
    
    # Remove hashtags and tabs from the tweet text
    text = re.sub(r'#|\t\s|\t', '', text)
    
    # Remove non-alphanumeric characters
    text = re.sub(r'[^\w\s]', '', text)
    
    # Remove punctuation from the tweet text
    text = text.translate(str.maketrans('', '', string.punctuation))
    
    # Remove stop words from the tweet text
    stop_words = set(stopwords.words('english'))
    
    # Tokenizing and filtering the tweet text
    word_tokens = nltk.word_tokenize(text)
    filtered_text = [word for word in word_tokens if word not in stop_words]
    
    # Spell correction
    spell = SpellChecker()
    corrected_text = []
    for word in filtered_text:
        corrected_word = spell.correction(word)
        if corrected_word is not None:
            corrected_text.append(corrected_word)
            
    # Removing non-English words
    english_words = set(words.words())
    english_text = [word for word in corrected_text if len(word) > 2 and word in english_words]
    
    # Lemmatize each word in the tweet text to reduce each word to its base form 
    lemmatizer = WordNetLemmatizer()
    lemmatized_text = [lemmatizer.lemmatize(word) for word in english_text]
    
    # Join the words back into a single string and return
    return ' '.join(lemmatized_text)

In [ ]:
#Applying the preprocess_text function to every tweet in the text column.
data['text'] = data['text'].apply(preprocess_text)

### Imputing values:
We have 0 and 4 as values in sentiment column. We change the value 4 to 1. Here 0 indicates 'Negative' and 1 indicates 'Positive'.

In [ ]:
data['sentiment'] = data['sentiment'].replace(4, 1)

### Cleansed Dataframe:
After performing pre-processing, transformations, and imputing, this is our cleansed dataframe:

In [ ]:
data.head()

In [ ]:
data.tail()

### About the dataframe:

In [ ]:
# Displaying shape
print('Shape of Data Frame:', data.shape)

### Highlighting features/predictors/target variables:

Features/Predictors:
- text

Target Variable:
- sentiment

In [ ]:
#Information about the dataframe
data.info()

### Displaying descriptive statistics:

In [ ]:
data.describe(include='all')

## Visualizations:

### Pie Chart of Sentiment Distribution:

In [ ]:
# Count the number of occurrences for each sentiment
sentiment_counts = data['sentiment'].value_counts()

# Plot the sentiment distribution as a pie chart
plt.figure(figsize=(8, 6))
plt.pie(sentiment_counts, labels=['Negative', 'Positive'], autopct='%1.1f%%', startangle=90)
plt.title('Sentiment Distribution')
plt.show()

We can observe that the dataframe has equal positive and negative sentiment texts.

### Word cloud of text:

In [ ]:
stop_words = set(stopwords.words('english'))

wordcloud = WordCloud(width = 800, height = 800, background_color ='white', stopwords = stop_words,
                      min_font_size = 10).generate(''.join(data['text'])+" ")
plt.figure(figsize = (8, 8)) 
plt.imshow(wordcloud) 
plt.axis("off") 
plt.tight_layout(pad = 0) 
plt.show() 

### Positive and Negative Sentiment Word Cloud

In [ ]:
from wordcloud import WordCloud

# Create a WordCloud object for positive sentiment
positive_text = data[data['sentiment'] == 1]['text'].values
positive_text = ' '.join(positive_text)
positive_wordcloud = WordCloud(background_color='white').generate(positive_text)

# Create a WordCloud object for negative sentiment
negative_text = data[data['sentiment'] == 0]['text'].values
negative_text = ' '.join(negative_text)
negative_wordcloud = WordCloud(background_color='white').generate(negative_text)

# Plot the word clouds
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.imshow(positive_wordcloud, interpolation='bilinear')
plt.title('Positive Sentiment Word Cloud')
plt.axis('off')

plt.subplot(1, 2, 2)
plt.imshow(negative_wordcloud, interpolation='bilinear')
plt.title('Negative Sentiment Word Cloud')
plt.axis('off')

plt.tight_layout()
plt.show()

### Top 10 Active Users:

In [ ]:
user_counts = data['user'].value_counts().head(10)

# Plot the user activity
plt.figure(figsize=(10, 6))
plt.bar(user_counts.index, user_counts.values)
plt.xlabel('User')
plt.ylabel('Count')
plt.title('Top 10 Active Users')
plt.xticks(rotation=45)
plt.show()

Code to calculate number of occurences of different words in all of the tweet texts.

In [ ]:
# Tokenize the text into words using NLTK
tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
data['tokens'] = data['text'].apply(lambda x: tokenizer.tokenize(x))

# Create a list of all the words in the dataset
all_words = [word for tokens in data['tokens'] for word in tokens]

# Count the frequency of each word using Counter
val_count = Counter(all_words)

# Visualise the top 10 occuring words in the tweet texts
val_counts= pd.DataFrame.from_dict(val_count, orient= 'index').reset_index()
val_counts= val_counts.rename(columns= {'index': 'Word', 0: 'Occurrence'})
val_counts.sort_values('Occurrence', ascending= False, inplace= True)
val_counts.head(10)

### Bar Plot of Top 30 Occurring words in the Tweet Texts:

In [ ]:
top_30 = val_counts.iloc[:30, :]

plt.figure(figsize= (20,8))
plt.bar(x= top_30.Word, height= top_30.Occurrence)
plt.title('Top 30 Occurring Words in the Tweet texts')
plt.xlabel('Word')
plt.ylabel('Occurrence')
plt.xticks(rotation=45)
plt.show()

### Splitting the dataset for training and testing:

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(data['text'], data['sentiment'], test_size=0.2, random_state=42)

### Vectorizing the training data:

In [ ]:
#Converting the text into a vector of numerical values.
# vectorizer = CountVectorizer()
# X_train = vectorizer.fit_transform(X_train)

# Create a TfidfVectorizer object
tfidf = TfidfVectorizer(stop_words='english', max_features=650, max_df=0.9, min_df=7, binary=True, 
                                   ngram_range=(1,2))

# Fit the vectorizer to the text data
tfidf.fit(X_train)

# Transform the text data using the fitted vectorizer
X_train_tfidf = tfidf.transform(X_train)
X_test_tfidf = tfidf.transform(X_test)

### Building the model:

In [ ]:
#Using the Multinomial Naive Bayes (MNB) algorithm for model building.
model_nb = MultinomialNB()
model_nb.fit(X_train_tfidf, y_train)

### Making initial prediction:

In [ ]:
# X_test = vectorizer.transform(X_test)
y_pred_nb = model_nb.predict(X_test_tfidf)

### Displaying initial prediction:

In [ ]:
print('Initial Prediction:')
print(y_pred_nb)

### Evaluating the model:

In [ ]:
# Evaluating the performance of the model using different metrics.
print("Classification Report: ")
print(classification_report(y_test, y_pred_nb))

In [ ]:
# Calculate the Confusion matrix for Naive Bayes model
conf_mat_nb = confusion_matrix(y_test, y_pred_nb)

cm_display_nb = ConfusionMatrixDisplay(confusion_matrix = conf_mat_nb, display_labels = [False, True])

cm_display_nb.plot()
plt.show()

In [ ]:
# Define a function to evaluate different models.
def metrics(y_test, y_pred):
    metrics = []
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = recall_score(y_test, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    r2 = r2_score(y_test, y_pred)
    metrics.append(accuracy)
    metrics.append(precision)
    metrics.append(recall)
    metrics.append(f1)
    metrics.append(rmse)
    metrics.append(r2)
    return metrics

In [ ]:
nb_metrics = metrics(y_test, y_pred_nb)

### Building different models:


# Logistic Regression

In [ ]:
# Create a LogisticRegression object
model_lr = LogisticRegression()

# Fit the model to the training data
model_lr.fit(X_train_tfidf, y_train)

# Make predictions on the test data
y_pred_lr = model_lr.predict(X_test_tfidf)

In [ ]:
# Evaluating the performance of the model using different metrics.
print("Classification Report: ")
print(classification_report(y_test, y_pred_lr))

In [ ]:
# Calculate the confusion matrix for Logistic regression model
conf_mat_lr = confusion_matrix(y_test, y_pred_lr)

cm_display_lr = ConfusionMatrixDisplay(confusion_matrix = conf_mat_lr, display_labels = [False, True])

cm_display_lr.plot()
plt.show()

In [ ]:
lr_metrics = metrics(y_test, y_pred_lr)

In [ ]:
#Hyperparameter Tuning the model
from sklearn.model_selection import GridSearchCV

# Define the hyperparameters and their potential values
param_grid = {
    'C': [0.1, 1, 10],
    'penalty': ['l1', 'l2']
}

# Create a LogisticRegression object
model_lr = LogisticRegression()

# Create a GridSearchCV object
grid_search = GridSearchCV(estimator=model_lr, param_grid=param_grid, cv=5, scoring='accuracy')

# Fit the grid search to the training data
grid_search.fit(X_train_tfidf, y_train)

# Get the best hyperparameters and model
best_params = grid_search.best_params_
best_model = grid_search.best_estimator_

# Print the best hyperparameters
print("Best Hyperparameters:", best_params)

# Make predictions on the test data using the best model
y_pred_lr_tuned = best_model.predict(X_test_tfidf)

# Calculate evaluation metrics using the best model
accuracy_tuned = accuracy_score(y_test, y_pred_lr_tuned)
precision_tuned = precision_score(y_test, y_pred_lr_tuned)
recall_tuned = recall_score(y_test, y_pred_lr_tuned)
f1_tuned = f1_score(y_test, y_pred_lr_tuned)

# Print the evaluation metrics
print("Accuracy (Tuned):", accuracy_tuned)
print("Precision (Tuned):", precision_tuned)
print("Recall (Tuned):", recall_tuned)
print("F1 Score (Tuned):", f1_tuned)


# Decision Tree Model

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay

# Create an instance of DecisionTreeClassifier
dt = DecisionTreeClassifier(max_depth=10, random_state=42)

# Train the Decision Tree model
dt_model = dt.fit(X_train_tfidf, y_train)

# Make predictions on the test data
y_pred_dt = dt_model.predict(X_test_tfidf)

In [ ]:
# Print the classification report
print("Classification Report:")
print(classification_report(y_test, y_pred_dt))

In [ ]:
# Calculate the Confusion matrix for Decision Tree model
conf_mat_dt = confusion_matrix(y_test, y_pred_dt)

# Create a ConfusionMatrixDisplay object for visualization
cm_display_dt = ConfusionMatrixDisplay(confusion_matrix=conf_mat_dt, display_labels=[False, True])

# Plot the confusion matrix
cm_display_dt.plot()
plt.show()

In [ ]:
dt_metrics = metrics(y_test, y_pred_dt)

# XGBoost Model

In [ ]:
import xgboost as xgb

#Create an instance of XGB Classifier
xg = xgb.XGBClassifier(learning_rate=0.50, max_depth=10, random_state=42) #based on the tuned parameters

# Train the XGBoost model
xg_model = xg.fit(X_train_tfidf, y_train)

# Make predictions on the test data
y_pred_xg  = xg_model.predict(X_test_tfidf)

In [ ]:
# Print the classification report.
print("Classification Report: ")
print(classification_report(y_test, y_pred_xg))

In [ ]:
# Calculate the Confusion matrix for Naive Bayes model
conf_mat_xg = confusion_matrix(y_test, y_pred_xg)

cm_display_xg = ConfusionMatrixDisplay(confusion_matrix = conf_mat_xg, display_labels = [False, True])

cm_display_xg.plot()
plt.show()

In [ ]:
xg_metrics = metrics(y_test, y_pred_xg)

# LSTM Model

In [44]:
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from keras_preprocessing.sequence import pad_sequences
from keras_preprocessing.text import Tokenizer
from keras.models import Sequential
import keras

In [45]:
# Hyperparameters of the model
vocab_size = 3000 
oov_tok = ''
embedding_dim = 100
max_length = 200
padding_type='post'
trunc_type='post'

# tokenize sentences
tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(X_train)
word_index = tokenizer.word_index

# convert train dataset to sequence and pad sequences
train_sequences = tokenizer.texts_to_sequences(X_train)
train_padded = pad_sequences(train_sequences, padding='post', maxlen=max_length)

# convert Test dataset to sequence and pad sequences
test_sequences = tokenizer.texts_to_sequences(X_test)
test_padded = pad_sequences(test_sequences, padding='post', maxlen=max_length)

In [46]:
# model initialization
model = keras.Sequential([
    keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    keras.layers.Bidirectional(keras.layers.LSTM(64)),
    keras.layers.Dense(24, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])
# compile model
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
# model summary
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 200, 100)          300000    
                                                                 
 bidirectional (Bidirectiona  (None, 128)              84480     
 l)                                                              
                                                                 
 dense (Dense)               (None, 24)                3096      
                                                                 
 dense_1 (Dense)             (None, 1)                 25        
                                                                 
Total params: 387,601
Trainable params: 387,601
Non-trainable params: 0
_________________________________________________________________


In [47]:
import tensorflow as tf
tf.config.run_functions_eagerly(True)

In [48]:
batch_size=64
model.fit(train_padded, y_train, epochs = 20, batch_size=batch_size, verbose = 'auto')

Epoch 1/20


C:\Users\HP\anaconda3\lib\site-packages\tensorflow\python\data\ops\structured_function.py:264: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


20000/20000 [==============================] - 18528s 926ms/step - loss: 0.4880 - accuracy: 0.7614
Epoch 2/20
20000/20000 [==============================] - 17098s 855ms/step - loss: 0.4678 - accuracy: 0.7734
Epoch 3/20
20000/20000 [==============================] - 17215s 861ms/step - loss: 0.4579 - accuracy: 0.7795
Epoch 4/20
20000/20000 [==============================] - 16833s 842ms/step - loss: 0.4497 - accuracy: 0.7843
Epoch 5/20
20000/20000 [==============================] - 16834s 842ms/step - loss: 0.4423 - accuracy: 0.7887
Epoch 6/20
20000/20000 [==============================] - 17370s 869ms/step - loss: 0.4347 - accuracy: 0.7933
Epoch 7/20
20000/20000 [==============================] - 17008s 850ms/step - loss: 0.4270 - accuracy: 0.7981
Epoch 8/20
20000/20000 [==============================] - 19481s 974ms/step - loss: 0.4192 - accuracy: 0.8028
Epoch 9/20
20000/20000 [==============================] - 17275s 864ms/step - loss: 0.4113 - accuracy: 0.8072
Epoch 10/20
20000/200

In [74]:
from tensorflow.keras.models import save_model

save_model(model, "lstm_model_1.h5")

In [76]:
from tensorflow.keras.models import load_model
# load model
model = load_model('lstm_model_1.h5')
# summarize model.
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 200, 100)          300000    
                                                                 
 bidirectional (Bidirectiona  (None, 128)              84480     
 l)                                                              
                                                                 
 dense (Dense)               (None, 24)                3096      
                                                                 
 dense_1 (Dense)             (None, 1)                 25        
                                                                 
Total params: 387,601
Trainable params: 387,601
Non-trainable params: 0
_________________________________________________________________


In [82]:
model.evaluate(test_padded, y_test)

C:\Users\HP\anaconda3\lib\site-packages\tensorflow\python\data\ops\structured_function.py:264: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


10000/10000 [==============================] - 2988s 299ms/step - loss: 0.5368 - accuracy: 0.7595


[0.5368435382843018, 0.7595124840736389]

In [78]:
prediction = model.predict(test_padded)
# Get labels based on probability 1 if p>= 0.5 else 0
pred_labels = []
for i in prediction:
    if i >= 0.5:
        pred_labels.append(1)
    else:
        pred_labels.append(0)
print("Accuracy of prediction on test set : ", accuracy_score(y_test,pred_labels))

C:\Users\HP\anaconda3\lib\site-packages\tensorflow\python\data\ops\structured_function.py:264: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


10000/10000 [==============================] - 2983s 298ms/step
Accuracy of prediction on test set :  0.7595125


In [83]:
precision = precision_score(y_test, pred_labels)
recall = recall_score(y_test, pred_labels)
f1 = recall_score(y_test, pred_labels)
print(precision)
print(recall)
print(f1)

0.751553580461016
0.7775970991738627
0.7775970991738627


In [84]:
lstm_metrics = metrics(y_test, pred_labels)

### Metrics table of all the models:

In [85]:
# Create a table which contain all the metrics of both the models.
metrics_table = {
        'Metric': ['Accuracy','Precision','Recall', 'F1Score', 'RMSE', 'R2'], 
        'Naive Bayes': nb_metrics,
        'Logistic Regression': lr_metrics, 'Decision Tree': dt_metrics,
        'XGBoost': xg_metrics,
        'LSTM' : lstm_metrics
        }

df_metrics = pd.DataFrame(metrics_table, columns = ['Metric', 'Naive Bayes', 'Logistic Regression','Decision Tree',
                                                    'XGBoost', 'LSTM'])
df_metrics

,Metric,Naive Bayes,Logistic Regression,XG Boost,LSTM
0,Accuracy,0.707156,0.720050,0.722956,0.759513
1,Precision,0.720574,0.700742,0.695744,0.751554
2,Recall,0.679757,0.771224,0.795572,0.777597
3,F1Score,0.679757,0.771224,0.795572,0.777597


We can observe here that LSTM model has the highest accuracy followed by XGBoost model.


### Test sentences for prediction using LSTM Model:

In [49]:
sents = ['the movie plot is terrible but it had good acting', 'I hope your day rocks as much as you do', 
         'I hate everyone', "I love you, but I don't love you", 'you are too good for this world', 
         'you look too innocent, but I dont believe you']

# Vectorizing the test sentences.
v = tfidf.transform(sents)

# Predicting sentiment on the test sentences.
prediction = xg_model.predict(v)
pred_labels = []

for i in prediction:
    if i >= 0.5:
        pred_labels.append(1)
    else:
        pred_labels.append(0)

for i in range(len(sents)):
    print(sents[i])
    if pred_labels[i] == 1:
        s = 'Positive'
    else:
        s = 'Negative'
    print("Predicted sentiment: ",s)

the movie plot is terrible but it had good acting
Predicted sentiment:  Positive
I hope your day rocks as much as you do
Predicted sentiment:  Positive
I hate everyone
Predicted sentiment:  Negative
I love you, but I don't love you
Predicted sentiment:  Positive
you are too good for this world
Predicted sentiment:  Positive
you look too innocent, but I dont believe you
Predicted sentiment:  Negative


### References:

https://pandas.pydata.org/docs/reference/frame.html

https://www.kaggle.com/datasets/kazanova/sentiment140

https://medium.com/analytics-vidhya/sentiment-analysis-on-amazon-reviews-using-tf-idf-approach-c5ab4c36e7a1

https://towardsdatascience.com/nlp-preprocessing-with-nltk-3c04ee00edc0

https://www.geeksforgeeks.org/generating-word-cloud-python/

https://www.w3schools.com/python/python_ml_confusion_matrix.asp

https://scikit-learn.org/

https://docs.python.org/3/library/re.html